# Customer Segmentation Analysis

## Introduction

The past several years have seen a sharp rise in e-commerce sales. Retail e-commerce sales reached $$3.53 trillion in 2019, and e-retail revenues are expected to reach $6.54 trillion by 2022, according to Statista. This huge increase suggests that consumers' purchasing habits have changed significantly. When compared to traditional sales, e-commerce has the distinct advantage that all transaction data, such as the goods, pricing, and shopping time, can be precisely recorded and saved. By grouping consumers into meaningful categories based on extensive transaction data, the firm may better understand the habits and preferences of its customers and meet their requirements more quickly.

Enhancing customer retention and corporate profitability requires an enterprise's capacity to recognize customer behavior and choose the right customer. Practical information about the product from the perspective of the client advantages anticipated gleaned from many connection between the business and its clients hasbeen taken prisoner. In order to increase client happiness and reduce operating expenses, combining comparable aspects of the Clients are vital. Furthermore, grouping clients also lessen the intricacy of the product's design. Consequently, comprehension Features of the customer allow the business to control the product organizing and creating goods that are appropriate for consumers. 



## Data and Sources
This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

The online shop that is the subject of this article is a registered non-store firm with 80 employees that is situated in the UK. The business was founded in 1981 and specializes in offering unusual presents for every occasion. The merchant used to take orders over the phone and mostly rely on direct mailing catalogs for many years. Just two years prior, the business established its own website and made the full transition to the internet. Since then, the business has maintained a consistent and healthy clientele from around Europe and the United Kingdom, and it has amassed a vast quantity of consumer data. Additionally, the business markets and sells its goods via Amazon.co.uk.

Table 1 displays the 8 variables that make up the customer transaction dataset that the merchant owns. The dataset comprises all of the transactions that happened in the years 2010 and 2011. There were a total of 22,190 legitimate transactions throughout that time frame, linked to 4381 valid unique postcodes. The dataset contains 406 830 instances (record rows) that correspond to these transactions, each of which represents a specific item that was a part of a transaction. It should be mentioned that the variable PostCode is crucial to the operation of the company since it gives crucial information that identifies and tracks each individual customer, allowing for some in-depth analysis to be conducted in the current research.

On average, each postcode is associated with five transactions, that is, each customer has purchased a product from the online retailer about once every2 months. 

In addition, only consumers from the United Kingdom are analysed. It is interesting to notice that the average number of distinct products (items) contained in each transaction occurring in 2011 was 18.3 ( = 406 830 / 22 190). This seems to suggest that many of the consumers of the business were organizational customers rather than individual customers. 


In [1]:
# import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as px
import re
import warnings
warnings.filterwarnings('ignore')


In [2]:
# load the data and print the first few rows
online_retail = pd.read_excel('Online Retail.xlsx')
online_retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Initial Exploration

+ Missing 25% of CustomerID: Since market/customer segmentation necessitates putting each individual client into a category, the absence of unique consumer identification might provide an issue.

+ Negative Unit Prices: It is uncommon to have negative UnitPrice, as this would mean a cash outflow to a company. it could be as a result of incorrect discount configuration, a refund, cancellation of orders or a bad-debt/write-off incurred by the business.

+ Potential data reversal: Incorrect data mapping or formatting during data import from another system may cause a reversal of signs and result in negative results. Further investigation is needed to understand the nature and determine the best way to manage such data reversal.

+ CustomerID column should be an object not float




In [3]:
# check summary statistics
online_retail.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [4]:
# check for data types and missing values
online_retail.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
# make a copy of the data 
online_retail_data = online_retail.copy()

### Necessary Functions That will be Needed

In [6]:
# define a function to filter a dataframe based on specific filter
def filter_records(df, column, criterion, operator = 'equal'):
    '''
    Ths function accepts a dataframe and filter it based on certain criteria or conditions
    
    df: Dataframe in question
    column: column of interest
    criterion: condition to filter on 
    operator: defines an extra condition to determine the required output
    
    return : return a dataframe having filterd the original 
    '''
    
    if operator == 'equal':
        return df[df[column] == criterion]
    elif operator == 'less':
        return df[df[column] <= criterion]
    elif operator == 'greater':
        return df[df[column] >= criterion]

In [7]:
# define a function to remove records based on certain criteria or conditions
def remove_records(df, column, criterion):
    '''
    Ths function accepts a dataframe and remove records based on certain criteria or conditions
    
    df: Dataframe in question
    column: column of interest
    criterion: condition to filter on 
    
    return : return a datframe having removed the rows of interest
    '''
    return df[df[column] != criterion]

## Data Preprocessing and Feature Engineering

### 1. InvoiceDate: Separate Date and Time information from InvoiceDate

The `InvoiceDate` column contains both date and time of the transaction. These data are separated into individual columns to facilitate future feature engineering and data manipulation. we would also create new other date time variables columns like month, day, year, weekend and weekdays

In [8]:
# split datatime  from InvoiceDate
online_retail_data['Date'] = online_retail_data.InvoiceDate.dt.date
online_retail_data['Time'] = online_retail_data.InvoiceDate.dt.time
online_retail_data['Year'] = online_retail_data.InvoiceDate.dt.year
online_retail_data['Month'] = online_retail_data.InvoiceDate.dt.month
online_retail_data['MonthName'] = online_retail_data.InvoiceDate.dt.month_name()
online_retail_data['WeedDay'] = online_retail_data.InvoiceDate.dt.weekday

# remove invoice data 
online_retail_data.drop(['InvoiceDate'], axis = 1, inplace = True)

# verify
online_retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,Year,Month,MonthName,WeedDay
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2


### 2. InvoiceNo: Extract Transaction status from `InvoiceNo`

InvoiceNo contains both transaction status (i.e. having a 'C' denotes cancelled transaction) and transaction identifier (e.g. unique invoice number). This information could be extracted to facilitate further feature engineering.


In [9]:
# separate order status ad invoice number from InvoiceNo
online_retail_data['OrderCategory'] = online_retail_data['InvoiceNo'].apply(lambda x: re.findall(r'[A-Z]', str(x)))\
                                      .apply(lambda x: pd.Series(x))
online_retail_data['InvoiceNum'] = online_retail_data['InvoiceNo'].apply(lambda x: re.findall(r'\d+', str(x)))\
                                      .apply(lambda x: pd.Series(x))

# remove InvoiceNo 
online_retail_data.drop(['InvoiceNo'], axis = 1, inplace = True)

# verify 
online_retail_data.head()

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,Year,Month,MonthName,WeedDay,OrderCategory,InvoiceNum
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2,NaN,536365
1,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2,NaN,536365
2,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2,NaN,536365
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2,NaN,536365
4,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2,NaN,536365


#### Drop Rows with Bad Debts
We have an A category and it indicate that the customer had a bad debt adjustments and does not represent actual sales and furthermore, they are not tagged to any specific customer so we will remove them. Nan means the orders from these customers were not cancelled

In [10]:
# check the unique order category
online_retail_data.OrderCategory.unique()

array([nan, 'C', 'A'], dtype=object)

In [11]:
# let filter for rows with the category A and check the description
online_retail_data[online_retail_data['OrderCategory'] == 'A']

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,Year,Month,MonthName,WeedDay,OrderCategory,InvoiceNum
299982,B,Adjust bad debt,1,11062.06,NaN,United Kingdom,2011-08-12,14:50:00,2011,8,August,4,A,563185
299983,B,Adjust bad debt,1,-11062.06,NaN,United Kingdom,2011-08-12,14:51:00,2011,8,August,4,A,563186
299984,B,Adjust bad debt,1,-11062.06,NaN,United Kingdom,2011-08-12,14:52:00,2011,8,August,4,A,563187


In [12]:
# remove records of interest
online_retail_data = remove_records(online_retail_data, 'OrderCategory', 'A')
#verify
online_retail_data.OrderCategory.unique()

array([nan, 'C'], dtype=object)

#### Create a new Column

We will create a new colum `cancelledOrder` to specify if an order was cancelled by changing the previous `OrderCategory` column to this and representing cancelled order with 1 and verified order with 0

In [13]:
# remane column 'Ordercategory' to 'CancelledOrder'
online_retail_data.rename(columns= {'OrderCategory':'CancelledOrder'}, inplace = True)

# fill Nan' with 0 and replace 'C' with 1
online_retail_data.CancelledOrder.fillna(value = 0, inplace=True)
online_retail_data.CancelledOrder.replace(to_replace = 'C', value = 1,
                                         inplace = True)

# veriify 
online_retail_data.CancelledOrder.unique()

array([0, 1], dtype=int64)

### 3. Unit Price


#### Remove records with unit price of 0
there are 2515 records where the unit prices are `zero` and it can be seen that the sales are not tagged to any customer. These transaction may be Small transactions that don't fit within the designated accounts in the ledgers and are referred to as miscellaneous costs. Costs that don't fit into a particular tax category are referred to as miscellaneous expenses, and they need to be tracked down and documented in the general ledger account of your company.

These records will be dropped as it has no relevance in our analysis. 

In [14]:
online_retail_data[online_retail_data.UnitPrice == 0]

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,Year,Month,MonthName,WeedDay,CancelledOrder,InvoiceNum
622,22139,NaN,56,0.0,NaN,United Kingdom,2010-12-01,11:52:00,2010,12,December,2,0,536414
1970,21134,NaN,1,0.0,NaN,United Kingdom,2010-12-01,14:32:00,2010,12,December,2,0,536545
1971,22145,NaN,1,0.0,NaN,United Kingdom,2010-12-01,14:33:00,2010,12,December,2,0,536546
1972,37509,NaN,1,0.0,NaN,United Kingdom,2010-12-01,14:33:00,2010,12,December,2,0,536547
1987,85226A,NaN,1,0.0,NaN,United Kingdom,2010-12-01,14:34:00,2010,12,December,2,0,536549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536981,72817,NaN,27,0.0,NaN,United Kingdom,2011-12-08,10:33:00,2011,12,December,3,0,581234
538504,46000M,POLYESTER FILLER PAD 45x45cm,240,0.0,NaN,United Kingdom,2011-12-08,13:58:00,2011,12,December,3,0,581406
538505,46000S,POLYESTER FILLER PAD 40x40cm,300,0.0,NaN,United Kingdom,2011-12-08,13:58:00,2011,12,December,3,0,581406
538554,85175,NaN,20,0.0,NaN,United Kingdom,2011-12-08,14:06:00,2011,12,December,3,0,581408


In [19]:
# remove records with UnitPrice as Zero
online_retail_data = remove_records(online_retail_data, 'UnitPrice', 0)

#verify 
filter_records(online_retail_data, 'UnitPrice', 0)

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,Year,Month,MonthName,WeedDay,CancelledOrder,InvoiceNum


#### New Column: `TotalPurchased`

We will create a new `TotalPurchased` by multipying the quantity purchased by the Unit price. It can still be seen that quantity had negative values so affecting the total sum also. 

In [27]:
# create a new column TotalPurchased
online_retail_data['TotalPurchased'] = online_retail_data['UnitPrice'] * online_retail_data['Quantity']

# verify 
online_retail_data.head()

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,Year,Month,MonthName,WeedDay,CancelledOrder,InvoiceNum,TotalPurchased
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2,0,536365,15.30
1,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2,0,536365,20.34
2,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2,0,536365,22.00
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2,0,536365,20.34
4,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,December,2,0,536365,20.34


## Quantity: with negative values

+ Product Returns or order cancellation: Product returns are frequently indicated by negative amounts. The quantity is recorded as a negative figure to reverse the transaction when buyers return things. In the event that the system records returns or refunds by reversing the initial transaction, this might potentially translate to negative unit costs.

+ Refunds: The reason you see these negative numbers might be explained if the system uses negative values in the quantity and unit pricing fields when processing refunds.

+ Reversed Transactions: In certain systems, a new transaction with the identical values as the original and negative signs to void the previous sale is recorded in response to adjustments or cancellations. Negative volumes and perhaps negative unit pricing would follow from this.
+ Stock Adjustments: Negative quantities might be used to adjust inventory levels when stock is found to be overcounted. This adjustment could reflect a correction rather than an actual sale or return, and might be paired with negative pricing in certain accounting or inventory systems.
+ Write-offs: If damaged or expired stock is written off, the system might use negative quantities to account for the reduction in inventory.

these are some of the reasons that could account for the neagtive quantities

In [31]:
filter_records(online_retail_data, 'Quantity',0, 'less')

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,Year,Month,MonthName,WeedDay,CancelledOrder,InvoiceNum,TotalPurchased
141,D,Discount,-1,27.50,14527.0,United Kingdom,2010-12-01,09:41:00,2010,12,December,2,1,536379,-27.50
154,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,4.65,15311.0,United Kingdom,2010-12-01,09:49:00,2010,12,December,2,1,536383,-4.65
235,22556,PLASTERS IN TIN CIRCUS PARADE,-12,1.65,17548.0,United Kingdom,2010-12-01,10:24:00,2010,12,December,2,1,536391,-19.80
236,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,0.29,17548.0,United Kingdom,2010-12-01,10:24:00,2010,12,December,2,1,536391,-6.96
237,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,0.29,17548.0,United Kingdom,2010-12-01,10:24:00,2010,12,December,2,1,536391,-6.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540449,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,0.83,14397.0,United Kingdom,2011-12-09,09:57:00,2011,12,December,4,1,581490,-9.13
541541,M,Manual,-1,224.69,15498.0,United Kingdom,2011-12-09,10:28:00,2011,12,December,4,1,581499,-224.69
541715,21258,VICTORIAN SEWING BOX LARGE,-5,10.95,15311.0,United Kingdom,2011-12-09,11:57:00,2011,12,December,4,1,581568,-54.75
541716,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,1.25,17315.0,United Kingdom,2011-12-09,11:58:00,2011,12,December,4,1,581569,-1.25


#### Remove records with maximum `Quantity = 80995`

Taking a closer look at the, we can see that the customer ID of the both transaction belong to the same person and occur on thhe same day. This means the person reversed or cancelled his order as can be seen from the table. Since the transaction was cancelled later, we will drop the recored that shows it was purchased.

In [28]:
online_retail_data.describe()

,Quantity,UnitPrice,CustomerID,Year,Month,WeedDay,CancelledOrder,TotalPurchased
count,539391.000000,539391.000000,406789.000000,539391.000000,539391.000000,539391.000000,539391.000000,539391.000000
mean,9.845921,4.653148,15287.795830,2010.921749,7.555805,2.432755,0.017219,18.092274
std,215.412851,93.409248,1713.573064,0.268567,3.508834,1.846509,0.130088,378.793696
min,-80995.000000,0.001000,12346.000000,2010.000000,1.000000,0.000000,0.000000,-168469.600000
25%,1.000000,1.250000,13954.000000,2011.000000,5.000000,1.000000,0.000000,3.750000
50%,3.000000,2.080000,15152.000000,2011.000000,8.000000,2.000000,0.000000,9.840000
75%,10.000000,4.130000,16791.000000,2011.000000,11.000000,4.000000,0.000000,17.400000
max,80995.000000,38970.000000,18287.000000,2011.000000,12.000000,6.000000,1.000000,168469.600000


In [32]:
filter_records(online_retail_data, 'Quantity', 80995)

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,Year,Month,MonthName,WeedDay,CancelledOrder,InvoiceNum,TotalPurchased
540421,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2.08,16446.0,United Kingdom,2011-12-09,09:15:00,2011,12,December,4,0,581483,168469.6


In [33]:
filter_records(online_retail_data, 'Quantity', -80995)

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,Year,Month,MonthName,WeedDay,CancelledOrder,InvoiceNum,TotalPurchased
540422,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2.08,16446.0,United Kingdom,2011-12-09,09:27:00,2011,12,December,4,1,581484,-168469.6


In [36]:
# remove the record with qith quantity ==80995
online_retail_data = remove_records(online_retail_data, 'Quantity', 80995)
#verify 
online_retail_data.Quantity.max()

74215

#### Remove records of previously purchases product and retain the records that shows the cancellation  
Now exploring further, It has been discovered that there are customers who bought products and later returned or cancelled the order and the records for both the purchase data and the returned or cancelled order data are retained. we will need to remove the records for the initially purchased record that were returned and retain the record that show the the cancelled order or returned product.  

In [37]:
filter_records(online_retail_data, 'Quantity', 74215)

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,Year,Month,MonthName,WeedDay,CancelledOrder,InvoiceNum,TotalPurchased
61619,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,1.04,12346.0,United Kingdom,2011-01-18,10:01:00,2011,1,January,1,0,541431,77183.6


In [38]:
filter_records(online_retail_data, 'Quantity', -74215)

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,Year,Month,MonthName,WeedDay,CancelledOrder,InvoiceNum,TotalPurchased
61624,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,1.04,12346.0,United Kingdom,2011-01-18,10:17:00,2011,1,January,1,1,541433,-77183.6
